In [6]:
import numpy
import datetime
from itertools import izip as zip, count
from nupic.encoders import ScalarEncoder
from nupic.encoders.random_distributed_scalar import RandomDistributedScalarEncoder
from nupic.encoders.date import DateEncoder
from nupic.encoders.category import CategoryEncoder
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory as TM

#nupicエンコーダーの活用(符号化)
#nupicのエンコーダーはHTMの入力用に異なるタイプ(データ)をSDR(疎な分散表現)にエンコードするためのクラス

#スカラーエンコーダー
#基本はn>=21

print "スカラーエンコーダー"
enc1=ScalarEncoder(n=22, w=3, minval=2.5, maxval=97.5, clipInput=True, forced=True)
print "3 =", enc1.encode(3)
print "4 =", enc1.encode(4)
print "5 =", enc1.encode(5)
#100が最大値になる
print "100  =", enc1.encode(100)
#100以上に値は同じ値にエンコードされる
print "1000 =", enc1.encode(1000)

print("--------------------------------------------------------------")

#例えば曜日をエンコードすると
#w = 3
#Minval = 1 #1週目のMonday
#Maxval = 8 #2週目のMonday
#periodic = true
#n = 14 #12時間単位で表せる(解像度)

print "スカラーエンコーダーの例"
enc2=ScalarEncoder(n=14, w=3, minval=1, maxval=8.0, periodic = True, forced=True)
print "monday (1)= ", enc2.encode(1)
print "sunday (7)=", enc2.encode(7)
#12時間の解像度なのでmonday midnight = Tuesday -> 01110000000000, monday noon -> 11100000000000
print "monday noon (1.5)= ", enc2.encode(1.5)

print("--------------------------------------------------------------")

#ランダム分散スカラーエンコーダー
#最大値と最小値のレンジを変化することができる

print "ランダム分散スカラーエンコーダー"
rdse = RandomDistributedScalarEncoder(n=21, w=3, resolution=5, offset=2.5)
print "3 =", rdse.encode(3)
print "4 =", rdse.encode(4)
print "5 =", rdse.encode(5)
print "100  =", rdse.encode(100)
print "1000 =", rdse.encode(1000)

print("--------------------------------------------------------------")

#日付エンコーダー
#デフォルトでは解像度が91.5日(1シーズン)になっている

print "日付エンコーダー"
de=DateEncoder(season=5)
now=datetime.datetime.strptime("2014-05-02 13:08:58", "%Y-%m-%d %H:%M:%S")
print "now =       ", de.encode(now)
nextMonth = datetime.datetime.strptime("2014-06-02 13:08:58", "%Y-%m-%d %H:%M:%S")
print "next month =", de.encode(nextMonth)
xmas = datetime.datetime.strptime("2014-12-25 13:08:58", "%Y-%m-%d %H:%M:%S")
print "xmas =      ", de.encode(xmas)

print("--------------------------------------------------------------")

#カテゴリーエンコーダー
#異なるものをエンコードするためのクラス

print "カテゴリーエンコーダー"
categories=("cat", "dog", "monkey", "slow loris")
encoder=CategoryEncoder(w=3, categoryList=categories, forced=True)
cat=encoder.encode("cat")
dog=encoder.encode("dog")
monkey=encoder.encode("monkey")
loris=encoder.encode("slow loris")
print "cat =       ", cat
print "dog =       ", dog
print "monkey =    ", monkey
print "slow loris =", loris

print "Noneやunknownが入力された場合"
print encoder.encode(None)
print encoder.encode("unknown")
print "cat([0 0 0 1 1 1 0 0 0 0 0 0 0 0 0])をデコードする"
print encoder.decode(cat)
#複雑な入力の時のデコード
#符号化された出力のビットを生成する範囲のリストが先頭で返される
catdog = numpy.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0])
print encoder.decode(catdog)


print("＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝")
#Spatial Pooler(空間プーリングの実装)
#空間プーリングに関しては詳細な計算アルゴリズムについてはまだ調べていない(読んでいない)が
#端的に言えばHebbian学習ルールと恒常性興奮制御の組み合わせを使用した任意のバイナリ入力パターンをSDRに変換する計算モデルである
#この空間プーリングによりニューロンのモデル化ができる

print "空間プーリングの実装"
#Spatial Poolerクラスは潜在的に接続できる範囲などのパラメーターを設定できる
sp=SpatialPooler(inputDimensions=(15,),columnDimensions=(4,),potentialRadius=15,numActiveColumnsPerInhArea=1,globalInhibition=True,synPermActiveInc=0.03,potentialPct=1.0)

print "Spatial Poolerのシナプス列を可視化する"
for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

print "Spatial Poolerでcat[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]を学習させる。アクティブな列のインデックスが返り値になる"
output = numpy.zeros((4,), dtype="int")
print "学習前"
print output
#computeするとアクティブな列のインデックスを出力することができる
sp.compute(cat, learn=True, activeArray=output)
print "学習後"
print output

#同じ学習を20回繰り返して計算する。この時outputの値は最終的に[0,0,1,0]になる
for _ in xrange(20):
    sp.compute(cat, learn=True, activeArray=output)

print "Spatial Poolerのシナプス列を再確認"
for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

#次にcat,dog,monkey,lorisすべてを学習させてみる
for _ in xrange(200):
    sp.compute(cat, learn=True, activeArray=output)
    sp.compute(dog, learn=True, activeArray=output)
    sp.compute(monkey, learn=True, activeArray=output)
    sp.compute(loris, learn=True, activeArray=output)

print "4つを学習した後のSpatial Poolerのシナプス列を確認する"
for column in xrange(4):
    connected2 = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected2)
    print connected2

noisyCat = numpy.zeros((15,), dtype="uint32")
noisyCat[3] = 1
noisyCat[4] = 1
noisyCat[6] = 1
print "catは[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]の列であるがノイズのかかった[0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]を入力する"
print "6列目はdogと学習した箇所である"
print noisyCat

print "ノイズのかかったnoisycatを学習させるとアクティブな列は前と同じになる = 4つ学習した後でノイズがかかった入力があっても学習した結果を思い出せた"
sp.compute(noisyCat, learn=False, activeArray=output)
print output  # matches cat!

print("＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝")
#Temporal Pooler(時空間メモリ,時間プーリング)の実装
#HTMではシーケンスメモリはTemporal Memoryアルゴリズムで実装されている
#Spatial Poolingによって形成されたSDRのシーケンスを学習して、次の入力SDRの予測を行うアルゴリズム
print "Temporal Memoryの実装"
#クラスTMは学習によるシナプスの増加量などを設定できる
tm = TM(columnDimensions = (50,),cellsPerColumn=2,initialPermanence=0.5,connectedPermanence=0.5,minThreshold=8,maxNewSynapseCount=20,permanenceIncrement=0.1,permanenceDecrement=0.0,activationThreshold=8)

#A -> B -> C -> D -> Eの流れを模した行列xを作成する
x = numpy.zeros((5, tm.numberOfColumns()), dtype="uint32")
x[0, 0:10] = 1    #"A"を表すSDRとする
x[1, 10:20] = 1   #"B"を表すSDRとする
x[2, 20:30] = 1   #"C"を表すSDRとする
x[3, 30:40] = 1   #"D"を表すSDRとする
x[4, 40:50] = 1   #"E"を表すSDRとする

print "tmにA,B,C,D,Eの5文字の学習を順に10回繰り返して行う。アクティブなセルの位置などの推移がわかる"
for i in range(10):
    #各文字を順番にTMに伝達する
    for j in range(5):
        #activeColumnsはset([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])～set([40, 41, 42, 43, 44, 45, 46, 47, 48, 49])を繰り返す
        activeColumns = set([i for i, j in zip(count(), x[j]) if j == 1])
        #print activeColumns
        #tm.computeは学習または推論を行うことができる(SPで学習と推論を行ったように)ここでは学習を行う
        tm.compute(activeColumns, learn = True)
        #tmの中身をトレースする際に使用する。
        print("active cells " + str(tm.getActiveCells()))
        print("predictive cells " + str(tm.getPredictiveCells()))
        print("winner cells " + str(tm.getWinnerCells()))
        print("# of active segments " + str(tm.connections.numSegments()))
        print("")
        #tm.reset()は必ずしも必要ではないが、シーケンスが終了したことをtmに伝えることができる。
    tm.reset()

#表示用なので無視してください
def formatRow(x):
    s = ''
    for c in range(len(x)):
        if c > 0 and c % 10 == 0:
            s += ' '
        s += str(x[c])
    s += ' '
    return s

print "元のベクトルを代入して元の結果が戻るかどうか確かめてみる。予測がきちんとできていることがわかる"
for j in range(5):
    print "\n\n--------","ABCDE"[j],"-----------"
    print "Raw input vector : " + formatRow(x[j])
    activeColumns = set([i for i, j in zip(count(), x[j]) if j == 1])
    tm.compute(activeColumns, learn = False)
    print "\nAll the active and predicted cells:"
    print("active cells " + str(tm.getActiveCells()))
    print("predictive cells " + str(tm.getPredictiveCells()))
    print("winner cells " + str(tm.getWinnerCells()))
    print("# of active segments " + str(tm.connections.numSegments()))
    activeColumnsIndeces = [tm.columnForCell(i) for i in tm.getActiveCells()]
    predictedColumnIndeces = [tm.columnForCell(i) for i in tm.getPredictiveCells()]
    actColState = ['1' if i in activeColumnsIndeces else '0' for i in range(tm.numberOfColumns())]
    actColStr = ("".join(actColState))
    predColState = ['1' if i in predictedColumnIndeces else '0' for i in range(tm.numberOfColumns())]
    predColStr = ("".join(predColState))
    print "Active columns:    " + formatRow(actColStr)
    print "Predicted columns: " + formatRow(predColStr)

スカラーエンコーダー
3 = [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
4 = [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
5 = [0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
100  = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
1000 = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
--------------------------------------------------------------
スカラーエンコーダーの例
monday (1)=  [1 1 0 0 0 0 0 0 0 0 0 0 0 1]
sunday (7)= [0 0 0 0 0 0 0 0 0 0 0 1 1 1]
monday noon (1.5)=  [1 1 1 0 0 0 0 0 0 0 0 0 0 0]
--------------------------------------------------------------
ランダム分散スカラーエンコーダー
3 = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
4 = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
5 = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1]
100  = [0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1000 = [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
--------------------------------------------------------------
日付エンコーダー
now =        [0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
next month = [0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
xmas =       [

active cells [61, 62, 64, 67, 69, 71, 72, 75, 76, 79]
predictive cells [81, 83, 84, 87, 89, 90, 93, 94, 96, 98]
winner cells [61, 62, 64, 67, 69, 71, 72, 75, 76, 79]
# of active segments 40

active cells [81, 83, 84, 87, 89, 90, 93, 94, 96, 98]
predictive cells []
winner cells [81, 83, 84, 87, 89, 90, 93, 94, 96, 98]
# of active segments 40

元のベクトルを代入して元の結果が戻るかどうか確かめてみる


-------- A -----------
Raw input vector : 1111111111 0000000000 0000000000 0000000000 0000000000 

All the active and predicted cells:
active cells [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
predictive cells [20, 23, 24, 27, 29, 31, 33, 35, 36, 39]
winner cells [0, 3, 4, 7, 9, 10, 12, 14, 17, 19]
# of active segments 40
Active columns:    1111111111 0000000000 0000000000 0000000000 0000000000 
Predicted columns: 0000000000 1111111111 0000000000 0000000000 0000000000 


-------- B -----------
Raw input vector : 0000000000 1111111111 0000000000 0000000000 0000000000 

All the active and predi